In [1]:
#Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import gmaps

#Import api key
from config import google_api_key

In [2]:
#Read cities csv
data_file = "../output_data/cities_Weather.csv"
cities_df = pd.read_csv(data_file)
cities_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity %,Cloudiness %,Wind Speed (mph)
0,Cherskiy,RU,68.75,161.30,8.11,91,22,8.12
1,Bengkulu,ID,-3.80,102.27,84.51,65,100,3.06
2,Ushuaia,AR,-54.80,-68.30,46.40,75,75,4.32
3,Inírida,CO,3.87,-67.92,74.14,93,69,1.99
4,Qaanaaq,GL,77.48,-69.36,-5.01,84,99,3.29


In [3]:
#Configuring gmaps
gmaps.configure(api_key=google_api_key)

#gmaps layout
layout = {'width':'900px', 'height':'500px', 'border':'1px solid black'}

coords = cities_df[["Latitude", "Longitude"]]
weight = cities_df["Humidity %"]
fig = gmaps.figure(layout=layout)
fig.add_layer(gmaps.heatmap_layer(coords, weights=weight))
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', width='900px'))

In [4]:
#Narrow our cities dataframe
#Cities with temp between 68 and 86 F
#Cities with less than 20% cloudiness
#Cities with wind speed less than 8 mph
narrowed_df = cities_df.loc[(cities_df["Temperature (F)"] >= 68) & (cities_df["Temperature (F)"] <= 86) &
                            (cities_df["Cloudiness %"] <= 20) & (cities_df["Wind Speed (mph)"] < 5)]
narrowed_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity %,Cloudiness %,Wind Speed (mph)
36,Morondava,MG,-20.28,44.28,81.70,68,0,4.81
58,The Valley,AI,18.22,-63.06,75.40,73,20,4.70
114,Doha,QA,25.29,51.53,73.69,43,0,4.70
161,Ixtapa,MX,20.70,-105.20,75.20,69,5,3.27
176,Sainte-Suzanne,RE,-20.91,55.61,78.93,74,4,4.70
179,Nizwá,OM,22.93,57.53,79.30,34,0,3.44
200,Dennery,LC,13.90,-60.89,75.00,75,2,1.01
224,Kavaratti,IN,10.57,72.64,86.00,74,20,4.70
249,Coahuayana Viejo,MX,18.73,-103.68,73.76,65,8,2.91
262,Abu Dhabi,AE,24.47,54.37,71.89,78,0,4.70


In [5]:
#Google places API url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Hotel names list
hotel_list = []

#Making a request for each city. Searching for the nearest hotel
for index, row in narrowed_df.iterrows():
    lat = row["Latitude"]
    long = row["Longitude"]
    location = f'{lat},{long}'
    params = {"key":google_api_key, "location":location, "radius":5000, "type":"lodging"}
    response = requests.get(base_url, params).json()
    print(f'Processing... {index}')
    try:
        hotel_list.append(response["results"][0]["name"])
    except:
        print("Error processing results")
        narrowed_df = narrowed_df.drop(labels=index)
narrowed_df["Hotel Name"] = hotel_list
narrowed_df

Processing... 36
Processing... 58
Processing... 114
Processing... 161
Processing... 176
Processing... 179
Processing... 200
Processing... 224
Processing... 249
Processing... 262
Processing... 294
Processing... 309
Processing... 356
Processing... 396
Processing... 402
Processing... 427
Processing... 454
Processing... 463
Processing... 494
Processing... 510
Processing... 529
Processing... 533
Processing... 542
Processing... 561
Processing... 562
Error processing results


,City,Country,Latitude,Longitude,Temperature (F),Humidity %,Cloudiness %,Wind Speed (mph),Hotel Name
36,Morondava,MG,-20.28,44.28,81.70,68,0,4.81,La Case Bambou
58,The Valley,AI,18.22,-63.06,75.40,73,20,4.70,CeBlue Villas & Beach Resort
114,Doha,QA,25.29,51.53,73.69,43,0,4.70,Ezdan Hotel
161,Ixtapa,MX,20.70,-105.20,75.20,69,5,3.27,Container Inn Puerto Vallarta
176,Sainte-Suzanne,RE,-20.91,55.61,78.93,74,4,4.70,Gites Ango
179,Nizwá,OM,22.93,57.53,79.30,34,0,3.44,Orient Apartments
200,Dennery,LC,13.90,-60.89,75.00,75,2,1.01,The Fox Grove Inn
224,Kavaratti,IN,10.57,72.64,86.00,74,20,4.70,Hotels in Lakshadweep Islands
249,Coahuayana Viejo,MX,18.73,-103.68,73.76,65,8,2.91,Hotel Los Arcos
262,Abu Dhabi,AE,24.47,54.37,71.89,78,0,4.70,Beach Rotana


In [6]:
#Drawing markers on map

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in narrowed_df.iterrows()]
coords = narrowed_df[["Latitude", "Longitude"]]

marker_layer = gmaps.marker_layer(coords, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', width='900px'))